In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sympy import sieve
import time
import tensorflow as tf
from itertools import combinations,permutations,product
from common import *

In [30]:
def which_one(links,steps):
    links_ = links[1:-1:2,np.newaxis]
    reverse_pos = 197769 - links[2:-1:2,np.newaxis]
    
    new_pos = (steps * offset_idx_from_node).sum(axis=-1) + offset_idx_from_pos + links[0]
    new_pos[:,:num_permu] = (new_pos[:,:num_permu] - links_)%10
    new_pos[:,num_permu:] = (new_pos[:,num_permu:] - reverse_pos)%10+10
    return new_pos

def connector_dist(connector,nodes):
    return distance(nodes[connector[:,::2]],nodes[connector[:,1::2]])

def segment_dist(which_offset,links,dist):
    segs = links[1:-1].reshape(opt-1,2)
    
    segs_penalty = []
    for a,b in segs:
        seg_penalty = dist[:,b] - dist[:,a]
        segs_penalty.append(seg_penalty)
    segment = sum([segs_penalty[i][which_offset[i]][which_middle_idx[i]] for i in range(len(segs))])
    
    return segment

def pair_step(links):
    return links[2:-1:2] - links[1:-1:2]

def check(links,nodes,solution,prime_set,dist):
    links_node = solution[links]
    steps = pair_step(links)
    
    not_prime = np.array([not x in prime_set for x in links_node])
    mask_prime = not_prime[mask_p]
    mask_ten = np.zeros_like(mask_prime,dtype=int)
    mask_ten[:,[0,-1]] = links[[1,-1]]
    mask_ten[:,1:-1] = mask_t_pos + (mask_t_node*steps).sum(axis=-1) + links[1]
    mask_ten = (mask_ten%10)==0
    mask = np.logical_and(mask_prime,mask_ten)*0.1
    
    connector = links_node[reconstruct]
    connector = connector_dist(connector,nodes)
    
    connector = (connector+connector*mask).sum(axis=-1)
    
    which_offset = which_one(links,steps)
    segment = segment_dist(which_offset,links,dist)
    
    out = connector+segment
    return np.argmin(out),out.min()-out[0]

def two_way_distance(nodes,solution,prime):
    s = nodes[solution[:-1]]
    e = nodes[solution[1:]]
    raw_d = distance(s,e)
    rever_raw_d = raw_d[::-1]
    
    n = len(raw_d)
    dist = np.zeros((20,n),dtype=float)
    zero_column = np.zeros((20,1),dtype=float)
    
    notprime = np.logical_not(np.isin(solution,prime))
    notprime_forward = notprime[:-1]
    notprime_backward = notprime[-1:0:-1]
    
    for offset in range(10):
        dist[offset,9-offset::10] += raw_d[9-offset::10] * notprime_forward [9-offset::10] * 0.1
        dist[offset+10,9-offset::10] += (rever_raw_d[9-offset::10] * notprime_backward[9-offset::10] * 0.1)
        dist[offset+10] = dist[offset+10][::-1]
    dist = np.concatenate([zero_column,dist],axis=-1)
    return dist.cumsum(axis=-1)

def excute_swap(code,links,solution):
    permu = permus[code//num_prod]
    prod = prods[code%num_prod]
    segs = links[1:-1].reshape(opt-1,2).copy()
    segs[:,1] += 1
    
    copies = []
    for i in permu:
        a,b = segs[i]
        copies.append(solution[a:b].copy())
    
    solution[segs[0,0]:segs[-1,-1]] = np.concatenate([copies[i][::prod[i]] for i in range(len(segs))])
    
    
def generate(opt):
    from itertools import combinations,permutations,product
    links = np.arange(opt*2)
    r = opt-1
    first,last = links[[0,-1]]
    pair = links[1:-1].reshape(r,2)
    
    pair_idx = list(range(r))
    pair_combo = []
    for _ in range(opt):
        pair_combo += list(combinations(pair_idx, _))
        
    num_possible_idx = len(pair_combo)
    possible_idx_from_pair = np.zeros((num_possible_idx,r),dtype=int)
    possible_idx_from_pos = np.zeros((num_possible_idx,),dtype=int)
    
    for i,pre in enumerate(pair_combo):
        pre_pairs_idx = list(pre)
        possible_idx_from_pair[i,pre_pairs_idx] = 1
        possible_idx_from_pos[i] = len(pre_pairs_idx)
    
    
    
    permus = np.array(list(permutations(np.arange(r))))
    prods = np.array(list(product([1,-1],repeat = r)))
    
    permus_n = len(permus)
    prods_n = len(prods)
    
    mask_p = np.zeros((permus_n*prods_n,opt),dtype=int)
    mask_p[:,0] = 0
    
    which_connect = np.zeros((opt,permus_n*prods_n,2),dtype=int)
    which_mask_t = np.zeros((opt,permus_n*prods_n),dtype=int)
    
    mask_t_node = np.zeros((permus_n*prods_n,opt-2,r),dtype=int)
    mask_t_pos = np.zeros((permus_n*prods_n,opt-2),dtype=int)
    
    which_new_pos = np.zeros((r, permus_n),dtype=int)
    
    for i in range(opt-2):
        mask_t_pos[:,i] = i + 1
        
    offset_idx_from_node = np.zeros((r,permus_n*2,r),dtype=int)
    offset_idx_from_pos = np.zeros((r,permus_n*2),dtype=int)
    which_middle_idx = np.zeros((r,permus_n*prods_n),dtype=int)
    reconstruct = []
    for i,permu in enumerate(permus):
        
        middle = pair[[permu]]
        where = np.argsort(permu)
        
        for j,pro in enumerate(prods):
            
               
            middle_ = middle.copy()
            for k,flip in enumerate(pro):
                
                middle_[k] = middle_[k][::flip]
                mask_p[i*prods_n+j,k+1] = middle_[k][-1]
                
                actual_seg = permu[k]
                before_current = permu[:k]
                if k>0:
                    mask_t_node[i*prods_n+j,k-1,before_current] = 1
                if flip == 1:
                    which_middle_idx[actual_seg,i*prods_n+j] = i
                else:
                    which_middle_idx[actual_seg,i*prods_n+j] = i+permus_n
            
            row = np.concatenate([[first],middle_.ravel(),[last]])
            for w in range(opt):
                rr,cc = row[w*2:w*2+2]
                which_connect[w,i*prods_n+j] = [rr,cc]
            reconstruct.append(row)
            
        for k in range(r):
            actual_seg = permu[k]
            before_current = permu[:k]

            offset_idx_from_pos[actual_seg,i] = k+1
            offset_idx_from_pos[actual_seg,i+permus_n] = k+1

            offset_idx_from_node[actual_seg,i,[before_current]] = 1
            offset_idx_from_node[actual_seg,i+permus_n,[before_current]] = 1
        
    
        
        for k in range(opt):
            if k<opt-1:
                actual_seg = permu[k]
                before_current = np.sort(permu[:k]).tolist()
                before_current = tuple(before_current)
                which_combo = pair_combo.index(before_current)
                
                which_mask_t[k,i*prods_n:(i+1)*prods_n] = which_combo
                which_new_pos[actual_seg, i] = which_combo
            else:
                which_mask_t[k,i*prods_n:(i+1)*prods_n] = len(pair_combo)-1
    reconstruct = np.array(reconstruct)      
    
    return reconstruct,permus,prods,which_middle_idx,offset_idx_from_node,\
           offset_idx_from_pos,mask_p,mask_t_pos,mask_t_node,which_connect,\
           possible_idx_from_pair,possible_idx_from_pos,which_mask_t,which_new_pos


def getlinks(pivot, candidates, solution):
    each =9
    node1,node2 = solution[pivot:pivot+2]
    already = [node1]
    c1 = candidates[node1][:each].tolist()
    already += (c1)
    
    c2 = candidates[node2]
    c2 = c2[np.isin(c2,already,invert=True)][:each].tolist()
    already += (c2)
    
    c3 = candidates[c1].ravel()
    c3 = c3[np.isin(c3,already,invert=True)][:each].tolist()
    already += (c3)
    
    c4 = candidates[c2].ravel()
    c4 = c4[np.isin(c4,already,invert=True)][:each].tolist()
    
    nolast = solution[:-1]
    c1 = np.argwhere(np.isin(nolast,c1)).ravel()
    c2 = np.argwhere(np.isin(nolast,c2)).ravel()
    c3 = np.argwhere(np.isin(nolast,c3)).ravel()
    c4 = np.argwhere(np.isin(nolast,c4)).ravel()
    node =  np.array(list(product([pivot],c1,c2,c3,c4))).T
    
    node.sort(axis=0)
    node = np.repeat(node,2,axis=0)
    node[1::2] += 1
    return node

def getlinks2(pivot, candidates, solution):
    each = 10
    total = 15
    node1,node2 = solution[pivot:pivot+2]
    pool = [node1] + candidates[node1][:each].tolist() + candidates[node2][:each].tolist()
    pool = np.unique(np.array(pool))[:total]
    
    nolast = solution[:-1]
    pool = np.argwhere(np.isin(nolast,pool)).ravel()
    
    links =  np.array(list(combinations(pool,opt))).T
    links.sort(axis=0)
    links = np.repeat(links,2,axis=0)
    links[1::2] += 1
    return links

In [3]:
p = problem(base_path='./solution/base4_2.csv')
p.score(p.base,raw=False)

solution = p.base.copy()
nodes = p.nodes.copy()
prime = p.prime.copy()
prime_set = p.prime_set
n = len(nodes)
dist = two_way_distance(nodes,solution,prime)
notprime = np.isin(np.arange(n),prime,invert=True)

opt = 5
reconstruct,permus,prods,which_middle_idx,\
offset_idx_from_node,offset_idx_from_pos,\
mask_p,mask_t_pos,mask_t_node,which_connect,\
possible_idx_from_pair,possible_idx_from_pos,which_mask_t,which_new_pos= generate(opt)

num_permu = len(permus)
num_prod = len(prods)
num_reconstruct = len(reconstruct)

candidates = pd.read_csv('20nn.csv').values

D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:134: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.


In [4]:
try:
    sess.close()
except:
    pass

tf.reset_default_graph()
#constant
_nodes = tf.constant(nodes, dtype='float64', shape=(n,2))
_notprime = tf.constant(notprime, dtype='bool', shape=(n,)) 
#_neighbors 

_reconstruct = tf.constant(reconstruct.T, dtype='int32') 
#permus
#prods
_which_connect = tf.constant(which_connect, dtype='int32')
_which_middle_idx = tf.constant(which_middle_idx, dtype='int32')
_offset_idx_from_node = tf.constant(offset_idx_from_node[:,:num_permu,:,np.newaxis], dtype='int32') 
_offset_idx_from_pos = tf.constant(offset_idx_from_pos[:,:num_permu,np.newaxis], dtype='int32') 
_mask_p = tf.constant(mask_p.T, dtype='int32') 
_possible_idx_from_pair = tf.constant(possible_idx_from_pair[:,:,None], dtype='int32') 
_possible_idx_from_pos = tf.constant(possible_idx_from_pos[:,None], dtype='int32') 
_which_mask_t = tf.constant(which_mask_t, dtype='int32') 
_which_new_pos = tf.constant(which_new_pos, dtype='int32') 

#bookkeepers
_solution = tf.Variable(solution, dtype='int32', expected_shape=(n+1,))
_dist = tf.Variable(dist.T, dtype='float64', expected_shape=(n+1,20))

#place holder
_links = tf.placeholder(dtype='int32', shape=(opt*2,None))
_link = tf.placeholder(dtype='int32', shape=(opt*2,))
_code = tf.placeholder(dtype='int32')

#wip
_linksnode = tf.gather(_solution, _links)
_linksnodecoor = tf.gather(_nodes, _linksnode)
_pairstep = _links[2:-1:2] - _links[1:-1:2]
_linksprime = tf.gather(_notprime, _linksnode)

_coor1 = tf.stack([_linksnodecoor]*(opt*2),axis=0)
_coor2 = tf.stack([_linksnodecoor]*(opt*2),axis=1)
_connectorrawdist = (_coor1 - _coor2) ** 2
_connectorrawdist = tf.sqrt(tf.reduce_sum(_connectorrawdist, axis=-1))
_connectorrawdist = tf.gather_nd(_connectorrawdist, _which_connect)

_maskprime = tf.gather(_linksprime, _mask_p)

_tenpos = tf.reduce_sum(_possible_idx_from_pair*_pairstep , axis=1)\
               + _possible_idx_from_pos\
               + _links[0] + 1
_masktenpos = tf.equal(_tenpos%10, 0)
_maskten = tf.gather(_masktenpos, _which_mask_t)
        
_mask = tf.logical_and(_maskprime, _maskten)
_connectordist = tf.where(_mask, _connectorrawdist*1.1, _connectorrawdist)
_connectordist = tf.reduce_sum(_connectordist, axis=0)

#'''
##segment
_forwardat = _links[1:-1:2,None,:]
_backwardat = n - _links[2:-1:2,None,:]
#'''     
#_newpos = tf.gather(_tenpos,_which_new_pos)
#_forwardoffset = (_newpos - _forwardat)%10
#_backwardatoffset = (_newpos - _backwardat)%10 + 10

_possibleforoffset = (_tenpos-_forwardat)%10
_possiblebackoffset = (_tenpos-_backwardat)%10 + 10
_forwardoffset = tf.batch_gather(_possibleforoffset, _which_new_pos)
_backwardatoffset = tf.batch_gather(_possiblebackoffset,_which_new_pos)

_whichoffset = tf.concat([_forwardoffset,_backwardatoffset], axis=1)

_segpenalty = tf.gather(_dist,_links[2:-1:2]) - tf.gather(_dist,_links[1:-1:2])
#_segments = []
#for pair in range(opt-1):
    #_segpenaltyslot = tf.batch_gather(_segpenalty[pair], tf.transpose(_whichoffset[pair]))
    #_segpenaltypermu = tf.gather(_segpenaltyslot, _which_middle_idx[pair], axis=1)
    #_segments.append(_segpenaltypermu)
#_segments = tf.transpose(sum(_segments))

_legpenalty = tf.batch_gather(_segpenalty, tf.transpose(_whichoffset,[0,2,1] ))
_legpenalty = tf.transpose(_legpenalty,[0,2,1])
_legpenalty = tf.batch_gather(_legpenalty, _which_middle_idx)
_segments = tf.reduce_sum(_legpenalty, axis=0)


#final
_out = _segments + _connectordist
_out = _out - _out[0:1]
_result = tf.argmin(tf.reshape(_out, [-1]), output_type = 'int32')
_best = tf.reduce_min(_out)
#updates
_newsolution = tf.placeholder(shape=(n+1,), dtype='int32')
_newdist = tf.placeholder(shape=(n+1, 20), dtype='float64')

_updatesolution = tf.assign(_solution, _newsolution, name='updatesolution')
_updatedist = tf.assign(_dist, _newdist, name='updatedist')

In [5]:
sess = tf.Session()
sess.run(tf.initializers.global_variables())

In [ ]:
print(p.score(solution))
a=time.time()

swaps = 1
iteration = 1

check_call = 0
for iteration in range(1,5):
    print('#######################')
    print('iteration %s'%iteration)
    for i in range(0,n):
        links = getlinks(i, candidates, solution)
        if len(links)>0:
            batch = links.shape[1]

            out = sess.run(_out,feed_dict={_links:links})
            check_call += 1
            result = np.argmin(out)
            
            which_link = result%batch
            code = result//batch
            if code>0:
                improvement = out[code,which_link]
                swaps += 1
                link = links[:,which_link]
                excute_swap(code,link,solution)
                dist = two_way_distance(nodes,solution,prime)

                sess.run(['updatesolution','updatedist'],feed_dict={_newsolution:solution,
                                                                   _newdist:dist.T})
                print(link,code,improvement,p.score(solution))
        if i%5000 == 0:
            b = time.time()
            print(i,b-a)
            a = time.time()
    print(time.time()-a)

1516130.8428928768
#######################
iteration 1
0 0.13800787925720215
[     7      8      8      9 196876 196877 197732 197733 197733 197734] 336 -1.5454829672762571 1516129.2974099093
[    33     34     35     36     36     37     42     43 196824 196825] 32 -0.2445675912204024 1516129.052842318
[116 117 119 120 151 152 706 707 708 709] 68 -0.2865570085901794 1516128.7662853096
[124 125 125 126 128 129 134 135 691 692] 80 -0.40817586885312096 1516128.3581094407
[163 164 249 250 254 255 256 257 257 258] 306 -0.19660936646167926 1516128.1615000742
[251 252 339 340 340 341 387 388 403 404] 32 -0.036047757681480164 1516128.1254523166
